# Dokumentum Metadatok Kiértékelése - CourtRankRL Projekt

Ez a notebook a processed_docs.jsonl fájlban található dokumentum metadatokat elemzi. Az agents.md specifikáció alapján készített kiértékelési szempontokat vizsgálja a jelenlegi adatokkal.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path
from typing import Dict, Any
from datetime import datetime

# Plot stílus beállítása
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Projekt konfiguráció betöltése
import sys
project_root = Path(__file__).parent.parent
sys.path.insert(0, str(project_root))
from configs import config

print("CourtRankRL - Document Metadata Analysis")
print(f"Processed docs: {config.PROCESSED_DOCS_LIST}")
print(f"Chunks: {config.CHUNKS_JSONL}")

NameError: name '__file__' is not defined

## 1. Feldolgozott Dokumentumok Betöltése

A processed_docs.jsonl fájl betöltése a metadatok elemzéséhez.

In [ ]:
# Processed docs betöltése
processed_docs_file = config.PROCESSED_DOCS_LIST
df = None

print(f"Processed docs betöltése: {processed_docs_file}")

if processed_docs_file.exists():
    try:
        processed_docs_list = []
        with open(processed_docs_file, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    doc = json.loads(line.strip())
                    processed_docs_list.append(doc)
                except json.JSONDecodeError:
                    continue
        
        if processed_docs_list:
            df = pd.DataFrame(processed_docs_list)
            print(f"✅ Betöltött dokumentumok száma: {len(df)}")
            print(f"Oszlopok: {df.columns.tolist()}")
        else:
            print("⚠️ Nem találhatóak feldolgozható dokumentumok")
            df = None
    except Exception as e:
        print(f"❌ Hiba a dokumentumok betöltése során: {e}")
        df = None
else:
    print(f"❌ Processed docs fájl nem található: {processed_docs_file}")
    print("Futtassa a build pipeline-t először: uv run courtrankrl build")
    df = None

if df is not None and not df.empty:
    print(f"\nAdatok betöltve: {df.shape[0]} dokumentum, {df.shape[1]} oszlop")
    print(f"Első néhány sor:")
    display(df.head())
else:
    print("\n❌ Nincs adat az elemzéshez")

## 2. Alapvető Információk és Statisztikák

A dokumentum metadatok alapvető statisztikáinak elemzése.

In [ ]:
if df is not None and not df.empty:
    print("📊 Alapvető információk:")
    print(f"Dokumentumok száma: {len(df)}")
    print(f"Oszlopok: {df.columns.tolist()}")
    
    # Adattípusok
    print(f"\nAdattípusok:")
    df.info()
    
    # Leíró statisztikák
    print("\n📈 Leíró statisztikák (numerikus oszlopok):")
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        display(df[numeric_cols].describe())
    
    print("\n📋 Leíró statisztikák (kategorikus oszlopok):")
    categorical_cols = df.select_dtypes(include=['object']).columns
    if len(categorical_cols) > 0:
        display(df[categorical_cols].describe())
else:
    print("❌ Nincs adat az alapvető információk elemzéséhez")

## 3. Hiányzó Értékek Elemzése

A hiányzó értékek azonosítása és vizualizációja.

In [ ]:
if df is not None and not df.empty:
    print("🔍 Hiányzó értékek elemzése:")
    
    # Hiányzó értékek statisztikái
    missing_values = df.isnull().sum()
    missing_percent = (missing_values / len(df)) * 100
    missing_df = pd.DataFrame({'Darabszám': missing_values, 'Százalék': missing_percent})
    missing_df = missing_df[missing_df['Darabszám'] > 0].sort_values(by='Százalék', ascending=False)
    
    print("Hiányzó értékek oszloponként:")
    display(missing_df)
    
    # Hiányzó értékek vizualizációja
    plt.figure(figsize=(15, 8))
    sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
    plt.title('Hiányzó értékek eloszlása a dokumentum metadatokban')
    plt.show()
    
    # Kritikus hiányzó értékek
    critical_missing = missing_df[missing_df['Százalék'] > 50]  # 50% feletti hiány
    if not critical_missing.empty:
        print(f"⚠️ Kritikus hiányzó értékek (>50%): {len(critical_missing)} oszlop")
        display(critical_missing)
    else:
        print("✅ Nincsenek kritikus hiányzó értékek")
else:
    print("❌ Nincs adat a hiányzó értékek elemzéséhez")

## 4. Kategorikus Változók Elemzése

A bíróságok, jogterületek és egyéb kategorikus változók elemzése.

In [ ]:
def plot_top_categories(df, column_name, top_n=20):
    """Segédfüggvény a leggyakoribb kategóriák megjelenítésére."""
    if column_name not in df.columns:
        print(f"❌ '{column_name}' oszlop nem található")
        return
    
    counts = df[column_name].value_counts()
    print(f"\n'{column_name}' egyedi értékeinek száma: {counts.nunique()}")
    print(f"Leggyakoribb {top_n} érték:")
    display(counts.head(top_n))
    
    plt.figure(figsize=(12, 6))
    counts.head(top_n).plot(kind='bar')
    plt.title(f'Dokumentumok megoszlása - {column_name} (Top {top_n})')
    plt.xlabel(column_name)
    plt.ylabel('Dokumentumok száma')
    plt.xticks(rotation=75, ha='right')
    plt.grid(axis='y')
    plt.tight_layout()
    plt.show()

if df is not None and not df.empty:
    print("📊 Kategorikus változók elemzése:")
    
    # Bíróság elemzése
    if 'birosag' in df.columns:
        plot_top_categories(df, 'birosag', top_n=30)
    
    # Jogterület elemzése
    if 'JogTerulet' in df.columns:
        plot_top_categories(df, 'JogTerulet', top_n=20)
    
    # Egyéb fontos kategorikus változók
    for col in ['MeghozoBirosag', 'Kollegium', 'AllKapcsolodoBirosag']:
        if col in df.columns:
            plot_top_categories(df, col, top_n=15)
else:
    print("❌ Nincs adat a kategorikus változók elemzéséhez")

## 5. Időbeli Elemzés

A határozatok időbeli eloszlásának elemzése.

In [ ]:
if df is not None and not df.empty and 'HatarozatEve' in df.columns:
    print("📅 Időbeli elemzés:")
    
    # Évszámok tisztítása
    df['HatarozatEve_clean'] = pd.to_numeric(df['HatarozatEve'], errors='coerce').astype('Int64')
    valid_years_mask = (df['HatarozatEve_clean'] >= 2005) & (df['HatarozatEve_clean'] <= pd.Timestamp.now().year)
    valid_years = df.loc[valid_years_mask, 'HatarozatEve_clean']
    
    print(f"Érvényes 'HatarozatEve' értékek száma (2005 óta): {len(valid_years)}")
    print(f"Érvénytelen vagy hiányzó 'HatarozatEve' értékek száma: {len(df) - len(valid_years)}")
    
    if not valid_years.empty:
        # Éves eloszlás
        year_counts = valid_years.value_counts().sort_index()
        print(f"\nÉvek tartomány: {year_counts.index.min()} - {year_counts.index.max()}")
        
        plt.figure(figsize=(15, 7))
        year_counts.plot(kind='line', marker='o', linewidth=2)
        plt.title('Dokumentumok megoszlása határozat éve szerint (2005 óta)')
        plt.xlabel('Határozat Éve')
        plt.ylabel('Dokumentumok száma')
        plt.grid(True, alpha=0.3)
        plt.fill_between(year_counts.index, year_counts.values, alpha=0.1)
        plt.show()
        
        # Top évek táblázata
        print("\nTop 10 év dokumentum szám szerint:")
        top_years = year_counts.nlargest(10)
        display(top_years.to_frame('Dokumentumok száma'))
    else:
        print("❌ Nincsenek érvényes évszámok 2005 óta")
else:
    print("❌ Nincs 'HatarozatEve' oszlop vagy adat az időbeli elemzéshez")

## 6. Bíróság és Jogterület Kapcsolata

A bíróságok és jogterületek közötti kapcsolat elemzése.

In [ ]:
if df is not None and not df.empty and 'birosag' in df.columns and 'JogTerulet' in df.columns:
    print("🔗 Bíróság és jogterület kapcsolata:")
    
    # Top bíróságok és jogterületek kiválasztása
    top_n_birosag = 20
    top_m_jogterulet = 15
    top_birosagok = df['birosag'].value_counts().nlargest(top_n_birosag).index
    top_jogteruletek = df['JogTerulet'].value_counts().nlargest(top_m_jogterulet).index
    
    # Szűrt adatkeret
    df_filtered = df[df['birosag'].isin(top_birosagok) & df['JogTerulet'].isin(top_jogteruletek)]
    print(f"Szűrt adatok: {len(df_filtered)} dokumentum")
    
    if not df_filtered.empty:
        # Kereszttábla létrehozása
        crosstab_bj = pd.crosstab(df_filtered['birosag'], df_filtered['JogTerulet'])
        
        # Heatmap
        plt.figure(figsize=(15, 10))
        sns.heatmap(crosstab_bj, cmap="viridis", annot=False, fmt="d", cbar_kws={'label': 'Dokumentumok száma'})
        plt.title(f'Bíróságok és jogterületek kapcsolata (Top {top_n_birosag} bíróság, Top {top_m_jogterulet} jogterület)')
        plt.xlabel('Jogterület')
        plt.ylabel('Bíróság')
        plt.xticks(rotation=60, ha='right')
        plt.yticks(rotation=0)
        plt.tight_layout()
        plt.show()
        
        # Legnagyobb kapcsolatok
        print("\nTop 20 bíróság-jogterület kapcsolat:")
        top_relations = crosstab_bj.stack().nlargest(20)
        display(top_relations.to_frame('Dokumentumok száma'))
    else:
        print("❌ Nem található elég adat a szűrt bíróságokhoz és jogterületekhez")
else:
    print("❌ Hiányzó oszlopok ('birosag' vagy 'JogTerulet') a kapcsolat elemzéséhez")

## 7. Chunk Számok Elemzése

A dokumentumokhoz tartozó chunk számok elemzése.

In [ ]:
if df is not None and not df.empty and 'chunk_count' in df.columns:
    print("📄 Chunk számok elemzése:")
    
    chunk_stats = df['chunk_count'].describe()
    print("Chunk számok statisztikái:")
    display(chunk_stats)
    
    # Chunk szám eloszlás
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    sns.histplot(x=df['chunk_count'], bins=50, kde=True)
    plt.title('Chunk számok eloszlása dokumentumonként')
    plt.xlabel('Chunkok száma')
    plt.ylabel('Dokumentumok száma')
    plt.grid(True, alpha=0.3)
    
    # Boxplot
    plt.subplot(1, 2, 2)
    sns.boxplot(y=df['chunk_count'])
    plt.title('Chunk számok boxplot')
    plt.ylabel('Chunkok száma')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Kiugró értékek
    q1, q3 = chunk_stats['25%'], chunk_stats['75%']
    iqr = q3 - q1
    upper_bound = q3 + 1.5 * iqr
    outliers = df[df['chunk_count'] > upper_bound]
    
    print(f"\nKiugró értékek (> {upper_bound:.1f} chunk): {len(outliers)} dokumentum")
    if len(outliers) > 0:
        print("Top 10 kiugró érték:")
        display(outliers[['doc_id', 'chunk_count']].nlargest(10, 'chunk_count'))
    
    # Chunk szám és év kapcsolata
    if 'HatarozatEve_clean' in df.columns:
        plt.figure(figsize=(12, 6))
        valid_data = df.dropna(subset=['chunk_count', 'HatarozatEve_clean'])
        if len(valid_data) > 100:
            plt.scatter(valid_data['HatarozatEve_clean'], valid_data['chunk_count'], alpha=0.5)
            plt.title('Chunk számok alakulása az évek során')
            plt.xlabel('Határozat éve')
            plt.ylabel('Chunkok száma')
            plt.grid(True, alpha=0.3)
            plt.show()
else:
    print("❌ Nincs 'chunk_count' oszlop a chunk számok elemzéséhez")

## 8. Azonosítók Elemzése

A különböző azonosítók (doc_id, Azonosito, EgyediAzonosito) elemzése.

In [ ]:
if df is not None and not df.empty:
    print("🆔 Azonosítók elemzése:")
    
    for id_col in ['doc_id', 'Azonosito', 'EgyediAzonosito']:
        if id_col in df.columns:
            print(f"\n🔍 '{id_col}' elemzése:")
            uniqueness = df[id_col].nunique()
            total_rows = len(df)
            duplicates = total_rows - uniqueness
            
            print(f"  Összes dokumentum: {total_rows}")
            print(f"  Egyedi értékek: {uniqueness}")
            print(f"  Duplikált/hiányzó értékek: {duplicates}")
            print(f"  Unikalitás aránya: {100 * uniqueness / total_rows:.2f}%")
            
            if duplicates > 0:
                duplicated_ids = df[df.duplicated(subset=[id_col], keep=False)][id_col].value_counts()
                print(f"  Leggyakoribb duplikált '{id_col}' értékek:")
                display(duplicated_ids[duplicated_ids > 1].head())
            
            # Mintavétel egyedi értékekből
            print(f"  '{id_col}' értékek mintái:")
            sample_ids = df[id_col].dropna().sample(min(5, uniqueness)).tolist()
            for i, sample_id in enumerate(sample_ids):
                print(f"    {i+1}. {sample_id}")
        else:
            print(f"\n⚠️ '{id_col}' oszlop nem található")
else:
    print("❌ Nincs adat az azonosítók elemzéséhez")

## 9. Jogszabályhelyek Mélyebb Elemzése

A jogszabályi hivatkozások elemzése.

In [ ]:
if df is not None and not df.empty and 'Jogszabalyhelyek' in df.columns:
    print("⚖️ Jogszabályhelyek elemzése:")
    
    # Diagnosztika
    print("\nDiagnosztika: Jogszabalyhelyek minta:")
    sample_values = df['Jogszabalyhelyek'].dropna().head(5)
    for i, val in enumerate(sample_values):
        print(f"  {i+1}: Type={type(val)}, Value='{str(val)[:100]}...'")
    
    # Jogszabályhelyek feldolgozása
    def parse_jogszabaly(value):
        if pd.isna(value) or not isinstance(value, str) or value.strip() == '[]' or not value.strip():
            return []
        try:
            # Egyszerű darabolás a '</br>' mentén
            items = value.split('</br>')
            # Üres elemek kiszűrése és szóközök eltávolítása
            cleaned_items = [item.strip() for item in items if item.strip()]
            return cleaned_items
        except Exception as e:
            print(f"Hiba a jogszabályhely feldolgozása közben: {e}")
            return []
    
    df['parsed_jogszabalyhelyek'] = df['Jogszabalyhelyek'].apply(parse_jogszabaly)
    jogszabaly_series = df['parsed_jogszabalyhelyek'].explode().dropna()
    
    print(f"\nÖsszesen {len(jogszabaly_series)} jogszabályhely hivatkozás található")
    print(f"Dokumentumok jogszabályhellyel: {df['parsed_jogszabalyhelyek'].apply(len).gt(0).sum()}/{len(df)}")
    
    if not jogszabaly_series.empty:
        jogszabaly_counts = jogszabaly_series.value_counts()
        print(f"\nEgyedi jogszabályhelyek száma: {len(jogszabaly_counts)}")
        
        # Top jogszabályhelyek
        top_n = 30
        print(f"\nLeggyakrabban hivatkozott {top_n} jogszabályhely:")
        display(jogszabaly_counts.head(top_n).to_frame('Hivatkozások száma'))
        
        # Vizualizáció
        plt.figure(figsize=(12, 8))
        jogszabaly_counts.head(top_n).plot(kind='barh')
        plt.title(f'Leggyakrabban hivatkozott {top_n} jogszabályhely')
        plt.xlabel('Hivatkozások száma')
        plt.ylabel('Jogszabályhely')
        plt.gca().invert_yaxis()
        plt.grid(axis='x', alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        # Jogszabály típus szerinti csoportosítás
        def get_law_type(jogszabaly):
            if 'törvény' in jogszabaly.lower() or 'tv.' in jogszabaly.lower():
                return 'Törvény'
            elif 'rendelet' in jogszabaly.lower() or 'korm.' in jogszabaly.lower():
                return 'Rendelet'
            elif 'pk' in jogszabaly.lower() or 'polgári' in jogszabaly.lower():
                return 'Polgári törvénykönyv'
            elif 'btk' in jogszabaly.lower() or 'büntető' in jogszabaly.lower():
                return 'Büntető törvénykönyv'
            else:
                return 'Egyéb'
        
        jogszabaly_types = jogszabaly_series.apply(get_law_type)
        type_counts = jogszabaly_types.value_counts()
        
        plt.figure(figsize=(10, 6))
        type_counts.plot(kind='pie', autopct='%1.1f%%', startangle=90)
        plt.title('Jogszabály típusok megoszlása')
        plt.ylabel('')
        plt.show()
    else:
        print("❌ Nem található érvényes jogszabályhely hivatkozás")
else:
    print("❌ Nincs 'Jogszabalyhelyek' oszlop a jogszabályhelyek elemzéséhez")

## 10. Következtetések

A dokumentum metadatok kiértékelésének összefoglalása.

In [ ]:
print("=== DOKUMENTUM METADATOK ELEMZÉS ÖSSZEFOGLALÓ ===")
print("\n✅ Sikeresen elemezve:")
if df is not None:
    print(f"   📄 Dokumentumok: {len(df)} db")
    print(f"   📊 Oszlopok: {len(df.columns)} db")
    
    # Hiányzó értékek összesítése
    missing_total = df.isnull().sum().sum()
    total_cells = df.shape[0] * df.shape[1]
    missing_ratio = missing_total / total_cells * 100
    print(f"   ❌ Hiányzó értékek: {missing_total}/{total_cells} ({missing_ratio:.2f}%)")
    
    # Unikalitás ellenőrzése
    unique_doc_ids = df['doc_id'].nunique() if 'doc_id' in df.columns else 0
    print(f"   🆔 Egyedi doc_id: {unique_doc_ids}/{len(df)} ({100 * unique_doc_ids / len(df):.2f}%)")

print("\n📋 Agents.md specifikáció ellenőrzés:")
if df is not None:
    # Alapvető metadatok megléte
    required_cols = ['doc_id', 'raw_path', 'chunk_count']
    missing_required = [col for col in required_cols if col not in df.columns]
    if not missing_required:
        print("   ✅ Alapvető metadatok jelen vannak")
    else:
        print(f"   ❌ Hiányzó alapvető metadatok: {missing_required}")
    
    # Időbeli adatok
    if 'HatarozatEve' in df.columns:
        valid_years = pd.to_numeric(df['HatarozatEve'], errors='coerce').notna().sum()
        print(f"   📅 Érvényes évszámok: {valid_years}/{len(df)} ({100 * valid_years / len(df):.2f}%)")
    
    # Jogterületek
    if 'JogTerulet' in df.columns:
        valid_domains = df['JogTerulet'].notna().sum()
        print(f"   ⚖️ Érvényes jogterületek: {valid_domains}/{len(df)} ({100 * valid_domains / len(df):.2f}%)")

print("\n💡 Ajánlások:")
if df is not None:
    # Kritikus hiányzó értékek
    critical_missing = df.isnull().sum()
    critical_cols = critical_missing[critical_missing > len(df) * 0.5]  # 50% feletti hiány
    if len(critical_cols) > 0:
        print(f"   🔧 Kritikus hiányzó értékek javítása: {list(critical_cols.index)}")
    
    # Duplikált azonosítók
    if 'doc_id' in df.columns:
        duplicates = len(df) - df['doc_id'].nunique()
        if duplicates > 0:
            print(f"   🆔 Duplikált doc_id-k javítása: {duplicates} db")
    
    # Hiányzó chunk számok
    if 'chunk_count' not in df.columns:
        print("   📝 Chunk számok hozzáadása szükséges")

print("\n🎯 Metadatok elemzése kész - a retrieval rendszer használatra kész!")